In [1]:
import numpy as np
import pandas as pd

from sklearn import datasets

#### Define the function for feature selection using similarity measure and fuzzy entroropy measures based on the article: 


In [2]:
def feature_selection_sim(data, measure = 'luca', p = 1):
    
    # OUTPUTS:
    # data_mod      data with removed feature
    # index_rem     index of removed feature in original data

    # INPUTS:
    # data          dataframe, contains class values
    # measure       fuzzy entropy measure, either 'luca' or 'park'              
    #               currently coded
    # p             parameter of Lukasiewicz similarity measure
    #               p in (0, \infty) as default p=1.
    
    # You need to import 'numpy' as 'np' before using this function

    l = int(max(data.iloc[:,-1]))   # -classes in the last column
    m = data.shape[0]               # -samples
    t = data.shape[1]-1             # -features
    
    dataold = data.copy()
    
    idealvec_s = np.zeros((l,t)) 
    for k in range(l):
        idx = data.iloc[:,-1] == k+1
        idealvec_s[k,:] = data[idx].iloc[:,:-1].mean(axis = 0)
    
    # scaling data between [0,1]
    data_v = data.iloc[:,:-1]
    data_c = data.iloc[:,-1] # labels
    mins_v = data_v.min(axis = 0)
    Ones   = np.ones((data_v.shape))
    data_v = data_v + np.dot(Ones,np.diag(abs(mins_v)))
    
    tmp =[]
    for k in range(l):
        tmp.append(abs(mins_v))
    
    idealvec_s = idealvec_s+tmp
    maxs_v     = data_v.max(axis = 0)
    data_v     = np.dot(data_v,np.diag(maxs_v**(-1)))
    tmp2 =[];
    for k in range(l):
        tmp2.append(abs(maxs_v))
        
    idealvec_s = idealvec_s/tmp2
    
    data_vv = pd.DataFrame(data_v) # Convert the array of feature to a dataframe
    data    = pd.concat([data_vv, data_c], axis=1, ignore_index=False)

    # sample data
    datalearn_s = data.iloc[:,:-1]
    
    # similarities
    sim = np.zeros((t,m,l))
    
    for j in range(m):
        for i in range(t):
            for k in range(l):
                sim[i,j,k] = (1-abs(idealvec_s[k,i]**p - datalearn_s.iloc[j,i])**p)**(1/p)
            
    sim = sim.reshape(t,m*l)
    
    # possibility for two different entropy measures
    if measure =='luca':
        # moodifying zero and one values of the similarity values to work with 
        # De Luca's entropy measure
        delta = 1e-10
        sim[sim == 1] = delta
        sim[sim == 0] = 1-delta
        H = (-sim*np.log(sim)-(1-sim)*np.log(1-sim)).sum(axis = 1)
    elif measure == 'park':
        H = (np.sin(np.pi/2*sim)+np.sin(np.pi/2*(1-sim))-1).sum(axis = 1) 
        
    # find maximum feature
    max_idx = np.argmax(H) # notice that index is starting from 0
    
    #removing feature from the data
    data_mod = dataold.drop(dataold.columns[max_idx], axis=1)
    
    return max_idx, data_mod


### Test the 'feature_selection_sim' function with an example dataset

In [3]:
# Initialization
p = 1
measure = 'luca'

data = [[0.4600,    0.3400,    0.1400,    0.0300,    0.9218,    1.0000],
        [0.5000,    0.3400,    0.1500,    0.0200,    0.7382,    1.0000],
        [0.4400,    0.2900,    0.1400,    0.0200,    0.1763,    1.0000],
        [0.7600,    0.3000,    0.6600,    0.2100,    0.4057,    2.0000],
        [0.4900,    0.2500,    0.4500,    0.1700,    0.9355,    2.0000],
        [0.7300,    0.2900,    0.6300,    0.1800,    0.9169,    2.0000]]

data = pd.DataFrame(data) # convert array data to a dataframe
    
idx, datanew = feature_selection_sim(data,measure,p) # function call

# idx: index of the removed feature
# datanew: data with removed feature

# data.columns[idx] # name of the removed feature


### Test the function with Iris dataset (you need to import datasets from from sklearn)

In [4]:
# Load the data
iris = datasets.load_iris()

# Load iris training dataset
X = iris.data

# Load iris target set
Y = iris.target

# Convert datasets' type into dataframe
df_f = pd.DataFrame(X, columns=iris.feature_names)
df_c = pd.DataFrame(Y)

data = pd.concat([df_f, df_c], axis=1, ignore_index=False)

idx, datanew = feature_selection_sim(data)

# data.columns[idx] : name of the removed feature


In [5]:
datanew.head()

,sepal length (cm),sepal width (cm),petal length (cm),0
0,5.1,3.5,1.4,0
1,4.9,3.0,1.4,0
2,4.7,3.2,1.3,0
3,4.6,3.1,1.5,0
4,5.0,3.6,1.4,0


In [6]:
datanew.to_csv("./processed_data.csv")

In [7]:
len(datanew)

150

In [6]:
Binavg = datanew.apply(lambda x: bin(hash(tuple(x))), axis = 1)
Binavg

0      0b11110000011010001001001001101001001101110111...
1      -0b1011001100000101011001100011000001001001111...
2      0b10000011101001100011111010010010011011101011...
3      -0b1001101000100110000101101111111101100101110...
4      0b11111111101000011110100010110010001101000101...
                             ...                        
145    -0b1110110011101101100011101110111011010111011...
146    0b11001110011010000100000101010001100001000000...
147    0b10010100010001110000101001011011000110011101...
148    -0b1010001000101101111110001010101011100011011...
149    0b10000111101100010011101100011001110101111011...
Length: 150, dtype: object

In [8]:
len(Binavg[0])

65